### 1. 데이터 물러오기

In [1]:
import pandas as pd
selloutData = pd.read_csv("../dataset/kopo_decision_tree_all_new.csv")
selloutData.head(5)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT
0,A01,PG01,P01,ITEM001,201538,2015,38,1,N,4,N,0.0
1,A01,PG01,P01,ITEM001,201548,2015,48,1,Y,0,N,0.0
2,A01,PG01,P01,ITEM001,201549,2015,49,2,Y,0,N,0.0
3,A01,PG01,P01,ITEM002,201526,2015,26,1,Y,1,N,0.0
4,A01,PG01,P01,ITEM002,201532,2015,32,1,N,4,N,0.0


### 2. 특성선정 및 데이터 분리

In [13]:
groupKey = ["REGIONID","PRODUCTGROUP","PRODUCT","ITEM"]

In [14]:
groupData = selloutData.groupby(groupKey)["YEARWEEK"].agg(["size"]).reset_index()

In [15]:
groupData.rename(columns={"size":"KNOB"},inplace=True)

In [19]:
len(groupData)

491

In [20]:
mergedData = pd.merge(left=selloutData,
         right=groupData,
         how="left",
         on=groupKey
        )

In [21]:
maxKnob = mergedData.KNOB.max()

In [34]:
cleansedData = mergedData[mergedData.KNOB >=  maxKnob ]
cleansedData.shape

(7592, 13)

In [9]:
cleansedData.dtypes

REGIONID         object
PRODUCTGROUP     object
PRODUCT          object
ITEM             object
YEARWEEK          int64
YEAR              int64
WEEK              int64
QTY               int64
HOLIDAY          object
HCLUS             int64
PROMOTION        object
PRO_PERCENT     float64
KNOB              int64
dtype: object

In [35]:
from sklearn.preprocessing import LabelEncoder
le_yn = LabelEncoder()
le_item = LabelEncoder()

In [36]:
cleansedData["LE_HOLI"] = le_yn.fit_transform(cleansedData.HOLIDAY)
cleansedData["LE_PROMO"] = le_yn.fit_transform(cleansedData.PROMOTION)
cleansedData["LE_ITEM"] = le_item.fit_transform(cleansedData.ITEM)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyd

### 3. 모델 생성

In [81]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn import linear_model
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import neighbors

In [118]:
# * 모델선언 딥러닝
model = Sequential()
model.add(Dense(units=8, activation="relu",input_shape=inputDim))
model.add(Dense(units=8, activation="relu"))
model.add(Dense(units=1, activation="relu"))
model.summary()

# * 모델선언 머신러닝
model_method_dt = tree.DecisionTreeRegressor(random_state =1)
model_method_lr = linear_model.LinearRegression()
model_method_rf = ensemble.RandomForestRegressor(random_state =1, n_estimators= 10)


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 8)                 40        
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 9         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


### 4. 모델 컴파일(훈련준비)

In [83]:
model.compile(loss='mean_squared_error', optimizer = 'adam', metrics=['mean_squared_error','mean_absolute_error'])

In [84]:
#callback List를 만들기 위한 선언 및 변수 설정
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from datetime import datetime
import os

log_folder = datetime.now().strftime("%Y%m%d_%H%M%S")

log_dirs = os.path.join("logs",log_folder)

### 5. 모델 훈련(callback 정의)

In [85]:
callback_list = [
    TensorBoard(log_dir = log_dirs),
    EarlyStopping(monitor="val_loss", patience=10), 
    ModelCheckpoint(filepath="./model_r_weights.h5", monitor="val_loss", verbose=1, save_best_only=True)   
]

In [113]:
def predict_deep(onegroup):

    eachgroup = onegroup.reset_index(drop=True)

    onegroup.head()
    corrdf = cleansedData.corr()
    corrdf

    features = list (corrdf[ (abs(corrdf.QTY) > 0.15) & (abs(corrdf.QTY) < 1)].index)
    if (len(features) > 0):
        
        features.append('LE_ITEM')
        label = ["QTY"]

        trainingData_features,testData_features,trainingData_label,testData_label,trainingData_all,testData_all = \
        train_test_split(eachgroup[features],eachgroup[label],eachgroup, test_size=0.2, random_state=1)

        inputDim = trainingData_features.loc[0,:].shape


        model.fit(x=trainingData_features,
                  y=trainingData_label,
                  batch_size=32,
                  epochs=600,
                  validation_split=0.2,
                  callbacks=callback_list)
        
        model_dt = model_method_dt.fit (trainingData_features ,trainingData_label)
        model_lr = model_method_lr.fit (trainingData_features ,trainingData_label)
        model_rf = model_method_rf.fit (trainingData_features ,trainingData_label)

        # * 4. 예측
        eachgroup["predict"] = model.predict(eachgroup[features])        
        eachgroup["predict_dt"] = model_dt.predict(eachgroup[features])
        eachgroup["predict_lr"] = model_lr.predict(eachgroup[features])
        eachgroup["predict_rf"] = model_rf.predict(eachgroup[features])
        eachgroup["PREDICT_YN"] = "Y"
    else:
        eachgroup["predict"] = 0
        eachgroup["predict_dt"] = 0
        eachgroup["predict_lr"] = 0
        eachgroup["predict_rf"] = 0
        eachgroup["PREDICT_YN"] = "N"
    
    return eachgroup

In [114]:
predictResult = cleansedData.groupby(groupKey).apply(predict_deep)

W1016 17:12:29.401478  3508 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Train on 92 samples, validate on 24 samples
Epoch 1/600
32/92 [=========>....................] - ETA: 0s - loss: 276004.0938 - mean_squared_error: 276004.0938 - mean_absolute_error: 256.3989
Epoch 00001: val_loss did not improve from 44.30626
92/92 [==============================] - 0s 318us/sample - loss: 471297.2147 - mean_squared_error: 471297.2188 - mean_absolute_error: 257.7108 - val_loss: 177425.1094 - val_mean_squared_error: 177425.1094 - val_mean_absolute_error: 262.8307
Epoch 2/600
32/92 [=========>....................] - ETA: 0s - loss: 348813.6562 - mean_squared_error: 348813.6562 - mean_absolute_error: 293.8541
Epoch 00002: val_loss did not improve from 44.30626
92/92 [==============================] - 0s 145us/sample - loss: 469803.7500 - mean_squared_error: 469803.7500 - mean_absolute_error: 256.5007 - val_loss: 176710.6875 - val_mean_squared_error: 176710.6875 - val_mean_absolute_error: 262.4167
Epoch 3/600
32/92 [=========>....................] - ETA: 0s - loss: 292402.

Epoch 20/600
32/92 [=========>....................] - ETA: 0s - loss: 19994.9863 - mean_squared_error: 19994.9863 - mean_absolute_error: 103.1239
Epoch 00020: val_loss did not improve from 44.30626
92/92 [==============================] - 0s 130us/sample - loss: 441814.7099 - mean_squared_error: 441814.6875 - mean_absolute_error: 257.9352 - val_loss: 174769.2500 - val_mean_squared_error: 174769.2500 - val_mean_absolute_error: 274.8208
Epoch 21/600
32/92 [=========>....................] - ETA: 0s - loss: 53944.8594 - mean_squared_error: 53944.8594 - mean_absolute_error: 163.9309
Epoch 00021: val_loss did not improve from 44.30626
92/92 [==============================] - 0s 162us/sample - loss: 440903.0462 - mean_squared_error: 440903.0312 - mean_absolute_error: 258.4096 - val_loss: 175063.6719 - val_mean_squared_error: 175063.6719 - val_mean_absolute_error: 275.6035


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
W1016 17:12:30.093894  3508 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>
W1016 17:12:30.123157  3508 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Train on 92 samples, validate on 24 samples
Epoch 1/600
32/92 [=========>....................] - ETA: 0s - loss: 304489.8125 - mean_squared_error: 304489.8125 - mean_absolute_error: 275.0425
Epoch 00001: val_loss did not improve from 44.30626
92/92 [==============================] - 0s 350us/sample - loss: 440398.7388 - mean_squared_error: 440398.7812 - mean_absolute_error: 259.1972 - val_loss: 175401.2031 - val_mean_squared_error: 175401.2031 - val_mean_absolute_error: 276.4930
Epoch 2/600
32/92 [=========>....................] - ETA: 0s - loss: 185713.7656 - mean_squared_error: 185713.7656 - mean_absolute_error: 191.0276
Epoch 00002: val_loss did not improve from 44.30626
92/92 [==============================] - 0s 146us/sample - loss: 439174.1848 - mean_squared_error: 439174.1875 - mean_absolute_error: 259.5154 - val_loss: 175723.0781 - val_mean_squared_error: 175723.0781 - val_mean_absolute_error: 277.2558
Epoch 3/600
32/92 [=========>....................] - ETA: 0s - loss: 170457.

...

Epoch 20/600
32/92 [=========>....................] - ETA: 0s - loss: 110503.0781 - mean_squared_error: 110503.0781 - mean_absolute_error: 258.9805
Epoch 00020: val_loss did not improve from 44.30626
92/92 [==============================] - 0s 226us/sample - loss: 154127.9494 - mean_squared_error: 154127.9531 - mean_absolute_error: 275.1523 - val_loss: 110692.3359 - val_mean_squared_error: 110692.3359 - val_mean_absolute_error: 263.4945
Epoch 21/600
32/92 [=========>....................] - ETA: 0s - loss: 187256.2812 - mean_squared_error: 187256.2812 - mean_absolute_error: 297.1924
Epoch 00021: val_loss did not improve from 44.30626
92/92 [==============================] - 0s 216us/sample - loss: 153418.5183 - mean_squared_error: 153418.5156 - mean_absolute_error: 275.4067 - val_loss: 111471.6641 - val_mean_squared_error: 111471.6641 - val_mean_absolute_error: 266.4248
Epoch 22/600
32/92 [=========>....................] - ETA: 0s - loss: 181188.6406 - mean_squared_error: 181188.6406 - 

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
W1016 17:14:27.346549  3508 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>
W1016 17:14:27.374324  3508 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Train on 92 samples, validate on 24 samples
Epoch 1/600
32/92 [=========>....................] - ETA: 0s - loss: 237664.6250 - mean_squared_error: 237664.6250 - mean_absolute_error: 435.7383
Epoch 00001: val_loss did not improve from 44.30626
92/92 [==============================] - 0s 318us/sample - loss: 259423.2147 - mean_squared_error: 259423.2188 - mean_absolute_error: 461.4090 - val_loss: 299855.7812 - val_mean_squared_error: 299855.7812 - val_mean_absolute_error: 497.0166
Epoch 2/600
32/92 [=========>....................] - ETA: 0s - loss: 192281.2656 - mean_squared_error: 192281.2656 - mean_absolute_error: 389.7253
Epoch 00002: val_loss did not improve from 44.30626
92/92 [==============================] - 0s 135us/sample - loss: 199215.5849 - mean_squared_error: 199215.5938 - mean_absolute_error: 392.5563 - val_loss: 206583.6250 - val_mean_squared_error: 206583.6250 - val_mean_absolute_error: 393.2531
Epoch 3/600
32/92 [=========>....................] - ETA: 0s - loss: 130325.

Epoch 20/600
32/92 [=========>....................] - ETA: 0s - loss: 30489.3926 - mean_squared_error: 30489.3926 - mean_absolute_error: 141.6469
Epoch 00020: val_loss did not improve from 44.30626
92/92 [==============================] - 0s 113us/sample - loss: 25541.4764 - mean_squared_error: 25541.4785 - mean_absolute_error: 127.1049 - val_loss: 29347.0684 - val_mean_squared_error: 29347.0684 - val_mean_absolute_error: 130.8759
Epoch 21/600
32/92 [=========>....................] - ETA: 0s - loss: 19948.5234 - mean_squared_error: 19948.5234 - mean_absolute_error: 111.2489
Epoch 00021: val_loss did not improve from 44.30626
92/92 [==============================] - 0s 108us/sample - loss: 25384.8099 - mean_squared_error: 25384.8105 - mean_absolute_error: 126.9855 - val_loss: 29586.1699 - val_mean_squared_error: 29586.1699 - val_mean_absolute_error: 131.5953
Epoch 22/600
32/92 [=========>....................] - ETA: 0s - loss: 24710.9160 - mean_squared_error: 24710.9160 - mean_absolute_

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
W1016 17:14:28.154035  3508 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


### 6. 모델 추론(validation)

In [115]:
finalResult = predictResult[predictResult.PREDICT_YN == "Y"]

In [117]:
mae_dnn = mean_absolute_error(finalResult["QTY"], finalResult.predict)
mae_dt = mean_absolute_error(finalResult["QTY"], finalResult.predict_dt)
mae_rf = mean_absolute_error(finalResult["QTY"], finalResult.predict_lr)
mae_lr = mean_absolute_error(finalResult["QTY"], finalResult.predict_rf)
dicMae = [ ["Deep Learning " ,mae_dnn ],["Decision Trees" , mae_dt],[ "Linear Regression" , mae_lr], ["Random Forests" ,mae_rf ]]
dicMae = sorted(dicMae, key=lambda t :t[1])
pd.DataFrame(dicMae)

,0,1
0,Decision Trees,54.470006
1,Linear Regression,56.068653
2,Random Forests,76.403407
3,Deep Learning,92.427597
